# Movie Recommendation System

Problem Definition:Creating a user-based movie rating prediction system by using KNN and weighted KNN algoritm.

In [39]:
import numpy as np
import os
import pandas as pd 
os.chdir(r"E:\Masaüstü\bbm409-assignment1") 

In [40]:
#importing needed data
mv=pd.read_csv("movies.csv").copy()
rt=pd.read_csv("ratings_train.csv").copy()
myTest=pd.read_csv("ratings_test_without_gt_new.csv")
myTest2=myTest[myTest["rating"]!=-1]#feature çıkartmak için convert edilmesi gereken test datası 
myTest=myTest[myTest["rating"]==-1]#predict edilmesi istenen test değerleri

In [43]:
###### combining movies.csv and ratings_train.csv with respect to movieıD
#combined data
new=pd.merge(rt, mv, on='movieId', how='inner')
new=pd.merge(rt, mv, on='movieId', how='inner')
new.head()


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,18,1,3.5,1455209816,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [44]:
#this method creates feature matrix for training data, returns training Data's feature matrix and trainingDataUsers matrix 
#parameters are combined data and movies.csv. Use only for training data
def dataConverter(new,mv):
    
    new["isAd"]=[1 if "Adventure" in i else 0 for i in new["genres"]]
    new["isAn"]=[1 if "Animation" in i else 0 for i in new["genres"]]
    new["isCh"]=[1 if "Children" in i else 0 for i in new["genres"]]
    new["isCo"]=[1 if "Comedy" in i else 0 for i in new["genres"]]
    new["isFa"]=[1 if "Fantasy" in i else 0 for i in new["genres"]]
    new["isRo"]=[1 if "Romance" in i else 0 for i in new["genres"]]
    new["isHo"]=[1 if "Horror" in i else 0 for i in new["genres"]]
    new["isAc"]=[1 if "Action" in i else 0 for i in new["genres"]]
    new["isDr"]=[1 if "Drama" in i else 0 for i in new["genres"]]
    new["isCr"]=[1 if "Crime" in i else 0 for i in new["genres"]]
    new["isTh"]=[1 if "Thriller" in i else 0 for i in new["genres"]]
    new["isMy"]=[1 if "Mystery" in i else 0 for i in new["genres"]]
    new["isSc"]=[1 if "Sci-Fi" in i else 0 for i in new["genres"]]
    new["isWa"]=[1 if "War" in i else 0 for i in new["genres"]]
    new["isDo"]=[1 if "Documentary" in i else 0 for i in new["genres"]]
    new2=new[new.columns[6:]].copy()
    new2["userId"]=new["userId"]
    trainingData=new2.groupby('userId').sum()
    trainingDataUsers=trainingData.index.to_numpy()
    trainingData=trainingData.to_numpy()
    return trainingData,trainingDataUsers

In [46]:
#this method creates feature matrix for test data, returns test Data's feature matrix and test DataUsers matrix 
#parameters are convert edilmesi gereken test datası(myTest2) and movies.csv. Use only for training data
def testDataConverter(new1,mv):
    new=new1[new1["rating"]!=-1]
    new1=new1[new1["rating"]==-1]
    new=pd.merge(new, mv, on='movieId', how='inner')
    new["isAd"]=[1 if "Adventure" in i else 0 for i in new.genres]
    new["isAn"]=[1 if "Animation" in i else 0 for i in new.genres]
    new["isCh"]=[1 if "Children" in i else 0 for i in new.genres]
    new["isCo"]=[1 if "Comedy" in i else 0 for i in new.genres]
    new["isFa"]=[1 if "Fantasy" in i else 0 for i in new.genres]
    new["isRo"]=[1 if "Romance" in i else 0 for i in new.genres]
    new["isHo"]=[1 if "Horror" in i else 0 for i in new.genres]
    new["isAc"]=[1 if "Action" in i else 0 for i in new.genres]
    new["isDr"]=[1 if "Drama" in i else 0 for i in new.genres]
    new["isCr"]=[1 if "Crime" in i else 0 for i in new.genres]
    new["isTh"]=[1 if "Thriller" in i else 0 for i in new.genres]
    new["isMy"]=[1 if "Mystery" in i else 0 for i in new.genres]
    new["isSc"]=[1 if "Sci-Fi" in i else 0 for i in new.genres]
    new["isWa"]=[1 if "War" in i else 0 for i in new.genres]
    new["isDo"]=[1 if "Documentary" in i else 0 for i in new.genres]
    new2=new[new.columns[5:]].copy()
    new2["userId"]=new["userId"]
    testData=new2.groupby('userId').sum()
    userIDVector=testData.index.to_numpy()
    testData=testData.to_numpy()
    return testData,userIDVector
    

In [47]:
#this method create and return a dataframe thats colums are moviesIds and rows are usersIds.Parameter is combined data.
#This basicly our known target
def trainTargetConverter(new):
    dic={}
    for i in new.index:
        if(new["userId"][i]  not in dic.keys()):
            dic[new["userId"][i]]={new["movieId"][i]:new["rating"][i]}
        else:
              dic[new["userId"][i]].update({new["movieId"][i]:new["rating"][i]})


    df = pd.DataFrame(dic) 
    df=df.T
    df=df.apply(lambda row: row.fillna(row.mean()), axis=1)
    df=df.sort_index()
    return df


In [48]:
#Assigning train trainUser test testUser
trainingData,trainingDataUsers=dataConverter(new,mv)
testData,testDataUsers=testDataConverter(myTest2,mv)
trainTarget=trainTargetConverter(new)

In [49]:
#Fit and Knn methods

from numpy import linalg as LA
def fitMethod(train,test,kPara,trainTarget,trainingDataUsers,testDataUsers):
    #cosine based similarity used
    similarityMatrix=((np.matmul(train, test.T))/(LA.norm(train)*LA.norm(test)))
    return KNNMethod(similarityMatrix,trainTarget,kPara,trainingDataUsers,testDataUsers)


#This method returns a dataframe that colums are all movies in the movies.csv and rows are testusersId
#return predictedValues
def KNNMethod(similarityMatrix,trainTarget,kPara,trainingDataUsers,testDataUsers):
    sm=similarityMatrix.copy()
    sm=pd.DataFrame(sm,index=trainingDataUsers,columns=testDataUsers)
    predictValues=[]
    dic={}
    for i in sm.columns:
        col=i
        sm=sm.sort_values(by=col, ascending=False)
        values=sm.index[0:kPara]
        dic[i]={x:(sum(trainTarget.loc[values,x])/len(values)) for x in trainTarget.columns}

    predictValues=pd.DataFrame(dic).T
   
    return predictValues

In [52]:
#this method calculates MAE.Parameters ara trainTarget and predictedValues and only for known target
def testMethod(trainTarget,predictValues):
    sumValue=0
    for i in predictValues.index:
        for y in predictValues.columns:
            sumValue=abs((trainTarget[y][i])-(predictValues[y][i]))+sumValue
        
    MAE=(1/(len(predictValues)*len(trainTarget.columns)))*sumValue
    return MAE

In [68]:
#crossValidation method is only for TESTING, it cant't be used in unknown targets.It returns average of number of error
def crossValidation(kFold,trainingData,kPara,trainTarget,trainingDataUsers):#trainingData is data ,k fold
    
    data=trainingData.copy()
    data2=trainingDataUsers.copy()
    folds = np.array_split(data, kFold)
    foldsUsers=np.array_split(data2,kFold)
    listOfAccu=[]
    for i in range(kFold):
        tempTrain=folds.copy()
        tempTrainUser=foldsUsers.copy()
        test=folds[i].copy()
        testDataUsers=foldsUsers[i].copy()
        del tempTrain[i]
        del tempTrainUser[i]
        train=np.concatenate(tempTrain, axis=0).copy()
        trainingDataUsers=np.concatenate(tempTrainUser, axis=0).copy()
        predictValues=fitMethod(train,test,kPara,trainTarget,trainingDataUsers,testDataUsers)
        listOfAccu.append(testMethod(trainTarget,predictValues))
    print("MAE for k-fold ",kFold," ",sum(listOfAccu)/kFold)  
    return sum(listOfAccu)/kFold

In [54]:
#predict method takes userId and Movie tuple or list and predictedValues as parameters return and print userId and rating
def predictMethod(userAndMovie,predictedValues):
    listOfPrediction=[]
    for i in userAndMovie:
        print(i[0],",",predictedValues[i[1]][i[0]])
        listOfPrediction.append([i[0],predictedValues[i[1]][i[0]]])
    return listOfprediction 

In [61]:
#less test data is used in this cell due to the time issues. k parameter is 3
predicted=fitMethod(trainingData[20:],trainingData[0:20],3,trainTarget,trainingDataUsers[20:],trainingDataUsers[0:20])

In [51]:
predicted.head()

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
4,3.666667,2.833333,2.966319,3.144321,2.04735,3.500000,2.833333,3.013669,2.763638,3.299652,...,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321
7,4.000000,2.833333,2.833333,2.960459,2.54735,3.449124,2.782457,2.829807,2.579776,3.500000,...,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459
10,3.666667,2.833333,2.966319,3.144321,2.04735,3.500000,2.833333,3.013669,2.763638,3.299652,...,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321
16,3.666667,2.833333,2.966319,3.144321,2.04735,3.500000,2.833333,3.013669,2.763638,3.299652,...,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321,3.144321
19,4.000000,2.833333,2.833333,2.960459,2.54735,3.449124,2.782457,2.829807,2.579776,3.500000,...,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459,2.960459


In [63]:
#MAE for k hyperparameter =3
MAE=testMethod(trainTarget,predicted)
print(MAE)

0.718745701427801


In [67]:
#trying different k parameter
for i in range(3,11):
    predicted=fitMethod(trainingData[20:],trainingData[0:20],i,trainTarget,trainingDataUsers[20:],trainingDataUsers[0:20])
    MAE=testMethod(trainTarget,predicted)
    print("MAE for ",i," hyperparameter",MAE)
    

MAE for  3  hyperparameter 0.718745701427801
MAE for  4  hyperparameter 0.7062708369633199
MAE for  5  hyperparameter 0.6368167278080952
MAE for  6  hyperparameter 0.6030906380597183
MAE for  7  hyperparameter 0.557272431223616
MAE for  8  hyperparameter 0.5363309773322302
MAE for  9  hyperparameter 0.5261050499817567
MAE for  10  hyperparameter 0.5329945908934768


In [69]:
#testing algoritm accuracy with cross validation kFold is 5 k parameter is 9.
#less training data is used in this test due to the time issue
predictedCv=crossValidation(5,trainingData[0:50],9,trainTarget,trainingDataUsers[0:50])

MAE for k-fold  5   0.4845092106479411


In [70]:
#for k=10 hyperparameter
predictedCv=crossValidation(5,trainingData[0:50],10,trainTarget,trainingDataUsers[0:50])

MAE for k-fold  5   0.48406924613027585


In [71]:
#weightedFitMethod and WeightedKnn method takes same parameters and return same values like normal fit and knn method.
#but in this knnMethod neighbors weight is like below
# (k is hyperparameter k) 1.neighbor*k+2.neighbor*k-1+....+k.neighbor*1
def weightedFitMethod(train,test,kPara,trainTarget,trainingDataUsers,testDataUsers):
    similarityMatrix=((np.matmul(train, test.T))/(LA.norm(train)*LA.norm(test)))
    return weightedKNNMethod(similarityMatrix,trainTarget,kPara,trainingDataUsers,testDataUsers)
    
def weightedKNNMethod(similarityMatrix,trainTarget,kPara,trainingDataUsers,testDataUsers):
    sm=similarityMatrix.copy()
    sm=pd.DataFrame(sm,index=trainingDataUsers,columns=testDataUsers)
    predictValues=[]
    dic={}
    kParaList=[k for k in range(kPara,0,-1)]
    sumKPara=sum(kParaList)
    for i in sm.columns:
        col=i
        sm=sm.sort_values(by=col, ascending=False)
        values=sm.index[0:kPara]
        mapped=zip(values,kParaList)
        mapped=[list(i) for i in mapped]
        dictionary={x[0]:x[1] for x in mapped}
        
        
        copyTrainTarget=trainTarget.loc[values,:].copy()
        
        copyTrainTarget=copyTrainTarget.mul(kParaList, axis=0)
            
        dic[i]={x:(sum(copyTrainTarget.loc[values,x])/sumKPara) for x in trainTarget.columns}
        
    
    predictValues=pd.DataFrame(dic).T
   
    return predictValues
    

In [72]:
for i in range(3,11):
    predictedWe=weightedFitMethod(trainingData[20:],trainingData[0:20],i,trainTarget,trainingDataUsers[20:],trainingDataUsers[0:20])
    MAE=testMethod(trainTarget,predictedWe)
    print("MAE for ",i," hyperparameter",MAE," in weighted KNN")

MAE for  3  hyperparameter 0.6990316897214891  in weighted KNN
MAE for  4  hyperparameter 0.6996309669354022  in weighted KNN
MAE for  5  hyperparameter 0.6771204180778738  in weighted KNN
MAE for  6  hyperparameter 0.6549371969429224  in weighted KNN
MAE for  7  hyperparameter 0.6292831653646285  in weighted KNN
MAE for  8  hyperparameter 0.6068984779981725  in weighted KNN
MAE for  9  hyperparameter 0.5895159628926648  in weighted KNN
MAE for  10  hyperparameter 0.5780313240983389  in weighted KNN
